In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import sqlite3
import mysql.connector

In [2]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',database='expression_atlas_shotgun', auth_plugin='mysql_native_password')
mycursor = conn.cursor()

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")


connection succesfull


1. Count number of proteins per tissue <br>
2. Number of assays per tissue <br>
3. number of unique peptides in the database <br>
4. number of proteotypic peptides in the database <br>
5. percentage of proteotypic peptides in the database <br>
6. amount of proteotypic peptides mapped to protein with at least three proteotypic peptides <br>
7. Percentage of proteotypic peptides used for the atlas <br>

In [3]:
## Number of projects
mycursor.execute("SELECT DISTINCT PXD_accession from project")
output_project = mycursor.fetchall()
output_project = list(sum(output_project, ()))
print('There are {} projects gathered in the database that do not contain Affinity purification data' .format(len(output_project)))
## Number of raw files
mycursor.execute("SELECT DISTINCT assay_id FROM assay")
output_assay = len(list(sum(mycursor.fetchall(), ())))
print('These projects contain a total of {} different raw files'.format(output_assay))
## Number of tissues and cell types
mycursor.execute("SELECT DISTINCT tissue_name FROM tissue")
output_tissue = len(list(sum(mycursor.fetchall(), ())))
mycursor.execute("SELECT DISTINCT cell_type FROM tissue")
output_cell = len(list(sum(mycursor.fetchall(), ())))
print('There are {} different tissues and {} different cell types found, all of them were manually annotated'.format(output_tissue, output_cell))
## Distribution diseased vs healthy
x = "SELECT * FROM tissue"
status = pd.read_sql_query(x, conn)
all_ = status.shape[0]
healthy = status[status['disease_status']=='Healthy']
healthy = healthy.shape[0]
diseased = all_ - healthy
print('From all the different samples (total of {}), {} of them contained healthy tissue and {} contained diseased/cancerous/cell culture/organoid tissue'.format(all_, healthy, diseased))

There are 217 projects gathered in the database that do not contain Affinity purification data
These projects contain a total of 11860 different raw files
There are 54 different tissues and 84 different cell types found, all of them were manually annotated
From all the different samples (total of 152), 75 of them contained healthy tissue and 77 contained diseased/cancerous/cell culture/organoid tissue


In [4]:
assayprojectsql = "SELECT * FROM assay"
assayprojectData = pd.read_sql_query(assayprojectsql, conn)
assayprojectData.head()

,assay_id,project_id,filename
0,27704,1756,HSE3
1,27705,1756,HSA
2,27706,1756,HSA3
3,27707,1756,HSE
4,27708,1756,HSB3


In [5]:
#2 number of assays per tissue
tissueassay = "SELECT * FROM tissue_to_assay"
tissueData = pd.read_sql_query(tissueassay, conn)
tissueData.head()

,assay_id,tissue_id
0,27704,452
1,27705,452
2,27706,452
3,27707,452
4,27708,452


In [6]:
tissueData_updated = pd.merge(tissueData, assayprojectData, on='assay_id', how='left')
tissueData_updated = tissueData_updated[['assay_id', 'tissue_id']]
print(tissueData.shape)
print(assayprojectData.shape)
print(tissueData_updated.shape)

(11860, 2)
(11860, 3)
(11860, 2)


In [7]:
assaycount = tissueData_updated.groupby(['tissue_id']).size()
assaycount = assaycount.to_frame()
assaycount = assaycount.rename(columns={0: 'assay_count'})

In [8]:
tissue_ids = "SELECT tissue_id, cell_type FROM tissue"
tissue_id_Data = pd.read_sql_query(tissue_ids, conn)
tissue_id_Data.shape

(152, 2)

In [9]:
final = pd.merge(assaycount, tissue_id_Data, on='tissue_id')

In [10]:
final = final.drop(columns=['tissue_id'])
final = final.groupby(['cell_type']).sum()
final.head(12)

,assay_count
cell_type,
Adipose tissue,14
Adrenalgland,3
Anterior chamber of the eye,80
Ascites,82
B-cells,27
Blood,604
Blood plasma,1932
Blood plasma (EDTA),20
Blood plasma (with heparin added),8


In [13]:
final.to_csv('assay_count_cells_shotgun.csv')

In [11]:
#3 number of unique peptides in the database
mycursor.execute("SELECT COUNT(DISTINCT(peptide_sequence)) FROM peptide")
unique_peptides = mycursor.fetchall()
x = unique_peptides[0]
total_unique_peptides = x[0]
print(total_unique_peptides)

234610


In [12]:
# 4. number of proteotypic peptides in the database
pepsql = "SELECT peptide_to_protein.peptide_id, peptide_to_protein.uniprot_id FROM peptide_to_protein"
pepData = pd.read_sql_query(pepsql, conn)
print(pepData.shape)
proteotypicData = pepData.groupby("peptide_id").filter(lambda x: len(x) == 1) #only retain peptides that are uniquely linked to one protein
print(proteotypicData.shape)
proteins = proteotypicData.groupby("uniprot_id").filter(lambda x: len(x) > 2) #only retain proteins that have at least three of those peptides
print(proteins.shape)

(234610, 2)
(234610, 2)
(229560, 2)


In [16]:
proteotypic_peptides = len(proteins['peptide_id'].unique())
proteotypic_peptides

229560

In [17]:
# percentage of proteotypic peptides in the database
percentage_proteotypic = (proteotypic_peptides/total_unique_peptides)*100
percentage_proteotypic

97.84749158177401

In [16]:
print('Before filtering: {} proteins found'.format(len(set(pepData['uniprot_id'].tolist()))))
print('After filtering: {} proteins found'.format(len(set(proteins['uniprot_id'].tolist()))))

Before filtering: 14779 proteins found
After filtering: 11259 proteins found


,peptide_id,uniprot_id
0,104622959,A0A024RBG1
1,105049495,A0A024RBG1
2,112818635,A0A024RBG1
3,105134601,A0A075B6H7
4,106273419,A0A075B6H7
...,...,...
234605,114274287,U3KPV4
234606,117706632,U3KPV4
234607,106731462,W5XKT8
234608,118402190,W5XKT8
